### ANALISIS 001 DE NTHUDDD

In [ ]:
import numpy as np
import os
import pandas as pd
import sklearn as sk
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm
import lightgbm as lgb
from joblib import load, dump
from sklearn.metrics import classification_report
from itertools import chain, combinations

metricas_analisis = [
    "frame_count", "perclos", "mean_ear", "current_time_closed_eyes", "blinks_per_minute",
    "yawns_per_minute", "head_nods_per_minute", "mean_blink_time", "head_nod", "yawn", "open_eyes", "drowsiness",
    "ear", "pitch", "yaw", "mar", "yawn_frequency", "head_nod_frequency", "blink_frequency"
]
# source_folder = "NTHUDDD_dataset2_step2_600/"
source_folder = "NTHUDDD_dataset2_nuevosmetodos/"
train_path = f"{source_folder}train/"
test_path = f"{source_folder}test/"
label = "drowsiness"
train_df_list = []
test_df_list = []
for filename in os.listdir(train_path):
    if "night" not in filename:
        file = os.path.join(train_path, filename)
        if os.path.isfile(file) and filename[-4:] == ".csv" and filename != "big_df.csv":
            df = pd.read_csv(file)[metricas_analisis]

            file_data = filename.split("_")
            df["subject"] = filename[:3]
            df["scenario"] = file_data[1]
            df["state"] = file_data[2][:-4]
            df["id"] = df['subject'] + df['scenario'] + df['state']
            train_df_list.append(df)

for filename in os.listdir(test_path):
    if "night" not in filename:
        file = os.path.join(test_path, filename)
        if os.path.isfile(file) and filename[-4:] == ".csv" and filename != "big_df.csv":
            df = pd.read_csv(file)[metricas_analisis]
            
            file_data = filename.split("_")
            df["subject"] = filename[:3]
            df["scenario"] = file_data[1]
            df["state"] = file_data[2][:-4]
            df["id"] = df['subject'] + df['scenario'] + df['state']
            test_df_list.append(df)
train_big_df = pd.concat(train_df_list)
test_big_df = pd.concat(test_df_list)
# big_df = big_df.drop("Unnamed: 0.1", axis=1)
# train_big_df.head()
test_big_df.head()

In [ ]:
# train_big_df.to_csv("NTHUDDD_dataset2/big_df.csv")

In [ ]:
filtered_train_big_df = train_big_df.copy()
filtered_test_big_df = test_big_df.copy()
# filtered_train_big_df = filtered_train_big_df[filtered_train_big_df["frame_count"] > 200]
filtered_train_big_df = filtered_train_big_df[filtered_train_big_df["scenario"] == "noglasses"]
filtered_train_big_df = filtered_train_big_df[filtered_train_big_df["subject"].isin(["001", "008", "009", "013", "023", "034", "035", "036"]) ]
# filtered_big_df = filtered_big_df[filtered_big_df["subject"].isin(["011"]) ]

# filtered_test_big_df = filtered_test_big_df[filtered_test_big_df["frame_count"] > 200]
filtered_test_big_df = filtered_test_big_df[filtered_test_big_df["scenario"] == "noglasses"]
# filtered_test_big_df = filtered_test_big_df[filtered_test_big_df["subject"].isin(["011", "016", "021"])]
# filtered_test_big_df = filtered_test_big_df[filtered_test_big_df["subject"].isin(["003", "010", "011", "014", "016", "017", "018", "019", "021", "025", "027", "028", "029", "037"])]
filtered_train_big_df.groupby(["subject", label]).mean()

#### NUM NULL ROWS

In [ ]:
print(f"TOTAL NUM ROWS: {filtered_train_big_df.shape[0]}")
print(f"NUM NULL ROWS: {filtered_train_big_df.shape[0] - filtered_train_big_df.dropna().shape[0]}")
train_big_df_nonull = filtered_train_big_df.dropna().sample(frac=1)
train_big_df_nonull["open_eyes"] = train_big_df_nonull["open_eyes"].astype("int")
train_big_df_nonull["head_nod"] = train_big_df_nonull["head_nod"].astype("int")
train_big_df_nonull["yawn"] = train_big_df_nonull["yawn"].astype("int")


print(f"TOTAL NUM ROWS: {filtered_test_big_df.shape[0]}")
print(f"NUM NULL ROWS: {filtered_test_big_df.shape[0] - filtered_test_big_df.dropna().shape[0]}")
test_big_df_nonull = filtered_test_big_df.dropna().sample(frac=1)
test_big_df_nonull["open_eyes"] = test_big_df_nonull["open_eyes"].astype("int")
test_big_df_nonull["head_nod"] = test_big_df_nonull["head_nod"].astype("int")
test_big_df_nonull["yawn"] = test_big_df_nonull["yawn"].astype("int")

#### Experimento 1: Exploracion del mejor conjunto de metricas

La idea es construir el experimento para que con una simple ejecución obtengamos un modelo por cada posible combinacion de metricas y sus respectivos resultados.

In [ ]:
metricas_posibles = [
    "perclos", "current_time_closed_eyes", "ear", "mar", "pitch",
    "yawns_per_minute", "head_nods_per_minute", "mean_blink_time"
]
experiment = False
tamano_minimo_combinacion = 3
tipo_modelo = "lgb"
experiment_folder = "model_experiments/experiment1/"
numero_metricas = len(metricas_posibles)
performance_results = []

combinacion_metricas = list(chain.from_iterable(combinations(metricas_posibles, size) for size in range(tamano_minimo_combinacion, numero_metricas + 1)))

In [ ]:
if experiment:
    for ind, combinacion in enumerate(combinacion_metricas):
        print(ind)
        # model_features = ["mean_blink_time", "perclos", "current_time_closed_eyes", "yawns_per_minute", "head_nods_per_minute"]
        model_features = list(combinacion)
        x_train = train_big_df_nonull[model_features]
        y_train = train_big_df_nonull[label]
        x_test = test_big_df_nonull[model_features]
        y_test = test_big_df_nonull[label]
        
        if tipo_modelo == "lgb":
            model = lgb.LGBMClassifier(
                boosting_type="dart",
                num_leaves=400,
                num_iterations=1000,
                learning_rate=0.01,
                verbosity=-1,
                #early_stopping=20,
            )
            file = os.path.join(experiment_folder, f"lgb_model_{ind}.joblib")
        elif mod == "svm":
            model = svm.SVC(kernel="linear")
            file = os.path.join(experiment_folder, f"svm_model_{ind}.joblib")
        
        model.fit(x_train, y_train)
        dump(model, file)
        y_pred = model.predict(x_train)
        train_res = classification_report(y_train, y_pred)
        y_pred2 = model.predict(x_test)
        test_res = classification_report(y_test, y_pred2)
        
        print(train_res)
        print(test_res)

        performance_results.append((model_features, train_res, test_res))

In [ ]:
if experiment:
    best_accuracy = -1
    best_ind = -1
    ind = 0
    for model_features, train_res, test_res in performance_results:
        # print(test_res)
        split_test_res = test_res.split()
        # print(split_test_res)
        accuracy = float(split_test_res[15])
        if accuracy >= best_accuracy:
            best_ind = ind
            best_accuracy = accuracy
        ind += 1

    print(f"BEST_IND: {best_ind} {performance_results[best_ind][0]} --> ACC: {best_accuracy}")

#### Escribimos los resultados

In [ ]:
if experiment:
    file_path = os.path.join(experiment_folder, "report.txt")
    if os.path.exists(file_path):
        os.remove(file_path)

    file = open(file_path, "a")
    for model_features, train_res, test_res in performance_results:
        file.write(f"{str(model_features)}\n")
        file.write(train_res)
        file.write(test_res)
        file.write("\n")
    file.close()

In [ ]:
model_features = ["yaw", "ear", "mar", "pitch", 'perclos', 'current_time_closed_eyes', "blinks_per_minute", "mean_blink_time", "yawns_per_minute", "head_nods_per_minute"]
x_train = train_big_df_nonull[model_features]
y_train = train_big_df_nonull[label]
x_test = test_big_df_nonull[model_features]
y_test = test_big_df_nonull[label]

x_test["yaw"] = abs(x_test["yaw"])

# test_size = len(labels)
# x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x_train)

x_train_normalized = min_max_scaler.transform(x_train)
x_test_normalized = min_max_scaler.transform(x_test)

In [ ]:
scaler = StandardScaler()
print(scaler.fit(x_train))
print(scaler.mean_)

x_train_transformed = scaler.transform(x_train)
x_test_transformed = scaler.transform(x_test)

In [ ]:
data_type = ""
x_train_data = x_train
x_test_data = x_test 
if data_type == "normalized":
    x_train_data = x_train_normalized
    x_test_data = x_test_normalized
elif data_type == "standardized":
    x_train_data = x_train_transformed
    x_test_data = x_test_normalized

mod = "lgb"
if mod == "lgb":
    model = lgb.LGBMClassifier(
        boosting_typ="gbdt",
        num_leaves=1820,
        num_iterations=1000,
        n_estimators=10000,
        max_depth=9,
        min_data_in_leaf=200,
        lambda_l1=100,
        lambda_l2=50,
        min_split_gain=6.23386152668858,
        bagging_fraction=0.2,
        baggin_freq=1,
        feature_fraction=0.9,
        learning_rate=0.2877383794743792,
        verbosity=-1,
        #early_stopping=20,
        importance_type='gain',
    )
    # model.fit(x_train_data, y_train, feature_name=model_features)
    file = "lgb_models/lgb_model_0.joblib"
elif mod == "svm":
    model = svm.SVC(kernel="linear")
    file = "svm_model_0.joblib"
elif mod == "knn":
    model = KNeighborsClassifier(n_neighbors=30)
    file = "knn_model_0.joblib"
elif mod == "reg":
    model = LogisticRegression(random_state=0, solver="saga", penalty="elasticnet", l1_ratio=0)
    file = "logreg_model_0.joblib"

# eval_set=[(x_test_data, y_test)], callbacks=[lgb.early_stopping(stopping_rounds=5)]
model.fit(x_train_data, y_train)
dump(model, file)
y_pred = model.predict(x_train_data)
print(classification_report(y_train, y_pred))
y_pred2 = model.predict(x_test_data)
print(classification_report(y_test, y_pred2))

In [ ]:
import time
s = time.time()
model.predict(x_test.iloc[5:105])
t = time.time() - s
print(t)
print(t / 100)

# print()

In [ ]:
print(y_pred2)

In [ ]:
optuna_model = load("lgb_model_optuna.joblib")
y_pred_proba = np.transpose(model.predict_proba(x_test))
print(y_pred_proba)
# y_pred = [ 0 if x0 > 0.5 else 1 for x0, x1 in y_pred_proba ]

In [ ]:
# y_pred2 = model.predict(x_test)
print(classification_report(y_test, y_pred2))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred2)

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_pred_proba[0], pos_label=0)

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_pred_proba[1], pos_label=1)

In [ ]:
PrecisionRecallDisplay.from_predictions(y_test, y_pred_proba[0], pos_label=0)

In [ ]:
PrecisionRecallDisplay.from_predictions(y_test, y_pred_proba[1], pos_label=1)

In [ ]:
import shap
import numpy as np
from joblib import load
# model = load("lgb_models/lgb_model_0.joblib")
# previous_model = load("lgb_model_003.joblib")
explainer = shap.TreeExplainer(model)

In [ ]:
y_pred2 = model.predict(x_test)
print(classification_report(y_test, y_pred2))

In [ ]:
x_importance = x_test
shap_values = explainer.shap_values(x_importance)
# print(len(shap_values))

In [ ]:
print(pd.DataFrame({'Value': model.feature_importances_, 'Feature': x_test.columns}))

In [ ]:
shap.summary_plot(shap_values[0], x_importance, plot_type="dot", plot_size=(14, 8))

In [ ]:
shap.dependence_plot("blink_frequency", shap_values[0], x_importance, interaction_index=None)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': x_test.columns})
plt.figure
plt.figure(figsize=(10, 10))
sns.set(font_scale = 5)
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                    ascending=False)[0:20])
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
# plt.savefig('lgbm_importances-01.png')
plt.show()

In [ ]:
row = 1311
pd_df = np.asarray(x_test.iloc[[row]])
# pd_df = np.array([[0.24, 0.08, (2/30), 0.0, 0]])
shap_values = explainer.shap_values(pd_df)
expected_value = explainer.expected_value

shap_values0 = shap_values[0]
shap_values1 = shap_values[1]
expected_value0 = expected_value[0]
expected_value1 = expected_value[1]

print(np.asarray(y_test.iloc[[row]]))
print(model.predict(pd_df))
print(shap_values)
print(expected_value)

In [ ]:
print(x_test.columns)
print(pd_df)
shap.summary_plot(shap_values0, features=pd_df, feature_names=x_test.columns, plot_type="dot")

In [ ]:
shap.decision_plot(expected_value0, shap_values0, features=pd_df, feature_names=list(x_test.columns))

#### Analisis error del modelo

In [ ]:
test_big_df_nonull.groupby(["subject", label]).mean()[metricas_posibles]

In [ ]:
# join test_data and predictions

test_with_preds_labels = test_big_df_nonull.copy()
test_with_preds_labels["pred"] = y_pred2
test_with_preds_labels["hit"] = test_with_preds_labels["pred"] == test_with_preds_labels[label]

print(test_with_preds_labels[label])
print(y_test)

test_with_preds_labels.groupby(["id", label])[["hit"]].mean()

#### Optuna hiperparameter tuning

In [ ]:
import optuna
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import log_loss
def train_lgb_optuna(x_train, y_train, x_test, y_test, features, save=False):
    def objective(trial, x_train, y_train, x_test, y_test):
        param_grid = {
            "device_type": trial.suggest_categorical("device_type", ['gpu']),
            "n_estimators": trial.suggest_categorical("n_estimators", [1000, 10000, 100000]),
            "learning_rate": trial.suggest_float("learning_rate", 0.0001, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
            "max_depth": trial.suggest_int("max_depth", 3, 40),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
            "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
            "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
            "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 25),
            "bagging_fraction": trial.suggest_float(
                "bagging_fraction", 0.2, 0.95, step=0.1
            ),
            "bagging_freq": trial.suggest_categorical("bagging_freq", [1, 2, 3]),
            "feature_fraction": trial.suggest_float(
                "feature_fraction", 0.2, 0.95, step=0.1
            ),
        }

        model = lgb.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            x_train,
            y_train,
            eval_set=[(x_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=300,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(x_test)
        score = log_loss(y_test, preds)
        print(score)
        return score

    study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
    func = lambda trial: objective(trial, x_train, y_train, x_test, y_test)
    study.optimize(func, n_trials=3000)

    best_params = {
        "objective": "binary",
        "metric": "accuracy",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "seed": 42
    } 
    best_params.update(study.best_params)
    print(f"\tBest value (binary_logloss): {study.best_value:.5f}")
    print(f"\tBest params:")

    for key, value in best_params.items():
        print(f"\t\t{key}: {value}")
    
    # x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

    train_dataset = lgb.Dataset(
        data=x_train,
        label=y_train,
    )
    lgb_model = lgb.train(best_params,
                          train_dataset,
                          num_boost_round=4000,
                          ) 

    y_pred = [round(pred) for pred in lgb_model.predict(x_train)]
    print(classification_report(y_train, y_pred))  
    y_pred = [round(pred) for pred in lgb_model.predict(x_test)]
    print(classification_report(y_test, y_pred))   

    if save:
        dump(lgb_model, "lgb_model_optuna.joblib")
    return lgb_model

In [ ]:
train_lgb_optuna(x_train, y_train, x_test, y_test, [], save=True)

In [ ]:
train_big_df_nonull.groupby(["subject", label]).mean()[metricas_posibles + ["open_eyes", "head_nod", "yawn"]]